In [6]:
from src.cv_test_pipeline import setup_equal_trees, split_dataset_into_folds
import pandas as pd

trees = setup_equal_trees(max_depth = 2, min_samples_leaf=20,tree_types = ['Majority','Trinary'])



In [7]:
# Set up variables
data_folder = (
    "/home/heza7322/PycharmProjects/missing-value-handling-in-carts/data/cleaned"
)
data_set = "titanic"
n_folds = 3

X = pd.read_csv(f"{data_folder}/{data_set}.csv", index_col=0)
y = X.pop("y")

folds = split_dataset_into_folds(X,y,n_folds)



In [ ]:
losses = {}
for tree in trees:
    y = pd.concat([folds[fold]["Test"][1] for fold in folds]).sort_index()
    if y.dtype == "object":
        y_prob = pd.DataFrame(columns=y.unique(), index=y.index, dtype="float")
    else:
        y_hat = pd.Series(index=y.index, dtype="float")
    for i, fold in folds.items():
        X_train, y_train = fold["Train"]
        X_test, _ = fold["Test"]

        tree.fit(X_train, y_train)
        if y.dtype == "object":
            y_prob.loc[X_test.index] = tree.predict(X_test, prob=True)
        else:
            y_hat.loc[X_test.index] = tree.predict(X_test)

    if y.dtype == "object":
        return calculate_loss(y=y, y_prob=y_prob)
    else:
        return calculate_loss(y=y, y_hat=y_hat)
